## Imports




In [86]:
import subprocess as sp
import glob

## Input

In [87]:
losspath = '../data/all_rodentlost_mirfams.list'
mirpath = '../data/hsa_mirgenedb_2_0.tsv'
rodmatdir = '../data/rodent_mature_mirnas'

## Read lost miRNA familiy info

In [92]:
def make_mirfam(raw):
    mirid = raw.split('_')[0]
    fam = '-'.join(mirid.split('-')[1:3])
    return mirid, fam


def read_mir(path):
    """
    returns: {mirfam: {mirid: matseq}, {mirid: matseq}, mirfam: ...}
    """
    resdict = {}
    with open(path, 'r') as fh:
        for line in fh:
            rawmir, chrom, start, end, strand, pre, mature = line.strip().split()
            mirid, mirfam = make_mirfam(rawmir)
            if mirfam == 'Mir-541':
                continue
            
            if not mirfam in resdict:
                resdict[mirfam] = {}
            resdict[mirfam][mirid] = mature.replace('T', 'U')
    return resdict


def filter_fam2seq(fam2seq, path):
    with open(losspath, 'r') as fh:
        lostmirs = [line for line in fh.read().split('\n') if line]
    lostfam2seq = {key: value for key, value in fam2seq.items() if key in lostmirs}
    return lostfam2seq

fam2seq = read_mir(mirpath)  
lostfam2seq = filter_fam2seq(fam2seq, losspath)
# {mirfam: {mirid: matseq}, {mirid: matseq}, mirfam: ...}

## Search for miRNA families that are inferred to be lost by ncOrtho but that are present in MirGeneDB

In [93]:
def FNs_from_fam(path, seqdict):
    fns = set()
    with open(path) as fh:
        for line in fh:
            if line.startswith('>'):
                _, dbfam = make_mirfam(line)
            if dbfam in seqdict:
                fns.add(dbfam)
    return list(fns)


for file in glob.glob(f'{rodmatdir}/*.fas'):
    ncOrtho_MirGeneDB_mismatch = FNs_from_fam(file, lostfam2seq)
    
print(ncOrtho_MirGeneDB_mismatch)

['Mir-489', 'Mir-506']


## Search for miRNAs in MirGeneDB with the same seed sequence 

In [94]:
def comp_seed(path, seqdict):
    lostseeds = {}
    for pair in seqdict.values():
        for mirid, lostmat in pair.items():
            lseed = lostmat[1:8]
            if mirid in lostseeds:
                raise ValueError('Seed mirid not unique')
            lostseeds[mirid] = lseed
    
    compseeds = set()
    with open(path) as fh:
        for line in fh:
            if line.startswith('>'):
                dbid = line.strip().replace('>', '')
            else:
                dbmature = line.strip()
                dbseed = dbmature[1:8]
                for mirid in lostseeds:
                    if lostseeds[mirid] == dbseed:
                        compseeds.add((dbid, mirid, dbseed))
                    
    return list(compseeds)


compseed = []
for file in glob.glob(f'{rodmatdir}/*.fas'):
    compseed.extend(comp_seed(file, lostfam2seq))
    
for comp in compseed:
    print(comp)

('Mmu-Mir-452-v1_5p', 'Hsa-Mir-506-P13', 'ACUGUUU')
('Mmu-Mir-506-P25_3p', 'Hsa-Mir-506-P11', 'ACUGUGU')
('Mmu-Mir-124-P2-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')
('Mmu-Mir-124-P3-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')
('Mmu-Mir-124-P1-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')
('Rno-Mir-489_3p', 'Hsa-Mir-489-v1', 'UGACAUC')
('Rno-Mir-506-P6_5p', 'Hsa-Mir-506-P6b', 'UCACAAG')
('Rno-Mir-452-v1_5p', 'Hsa-Mir-506-P13', 'ACUGUUU')
('Rno-Mir-124-P2-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')
('Rno-Mir-124-P1-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')
('Rno-Mir-124-P3-v2_3p', 'Hsa-Mir-506-P1', 'UAAGGCA')


## Search for significantly similar mature miRNA sequences

In [95]:
def make_blastndb(inpath, outpath):
    db_command = 'makeblastdb -in {} -out {} -dbtype nucl'.format(inpath, outpath)
    sp.call(db_command, shell=True)
    

def check_blastdb(db_path):
    file_extensions = ['nhr', 'nin', 'nsq']
    for fe in file_extensions:
        files = glob.glob(f'{db_path}*{fe}')
        if not files:
            # At least one of the BLAST db files is not existent
            return False
    return True


def run_blast(seq, blastdb, cpu=4, evalue=1):
    blast_command = (
                'blastn -task blastn-short -db {0} '
                '-num_threads {1} -evalue {2} '
                '-outfmt "6 saccver pident qcovs length evalue bitscore sseq"'.format(blastdb, cpu, evalue)
            )
    blast_call = sp.Popen(
        blast_command, shell=True, stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.PIPE, encoding='utf8'
    )
    res, err = blast_call.communicate(seq)
    if err:
        print(f'ERROR: {err}')
    return res


def blast2FNs(path, seqdict):
    if not check_blastdb(file):
        make_blastndb(path, path)
    FNs = []
    for mirfam in seqdict:
        for mirgene, mature in lostfam2seq[mirfam].items():
            #seed = mature[1:8]
            blastres = run_blast(mature, path, evalue=0.01)
            if blastres:
                _, blastfam = make_mirfam(blastres.split()[0])
                if blastfam == mirfam:
                    FNs.append(blastres.split()[0])
    return FNs



significantly_similar_mature_sequences = []
for file in glob.glob(f'{rodmatdir}/*.fas'):
    significantly_similar_mature_sequences.extend(blast2FNs(file, lostfam2seq))

print(ncOrtho_MirGeneDB_mismatch)
print(significantly_similar_mature_sequences)

['Mir-489', 'Mir-506']
['Mmu-Mir-489_3p', 'Mmu-Mir-489_3p', 'Rno-Mir-489_3p', 'Rno-Mir-489_3p', 'Rno-Mir-506-P23_5p', 'Rno-Mir-506-P6_5p']


### Conclusion

Potentially compensating miRNAs are all due to miRNAs that are inferred to be absent by ncOrtho, but have an entry in MirGeneDB